### SQLite 데이터 베이스와 Pandas 데이터프레임

In [ ]:
import sqlite3
import pandas as pd
import os

In [ ]:
from google.colab import drive
# 구글 드라이브 마운트.
drive.mount('/content/drive')  # 절차를 따라서 한다

In [ ]:
# 현 폴더의 위치를 보여준다.
!pwd        # Linux

In [ ]:
# 작업 폴더로 이동한다.
# 데이터 파일은 이미 올려져 있어야 한다.
os.chdir("/content/drive/MyDrive/GwangjuAI/notebook/")

In [ ]:
# 현 폴더의 위치를 보여준다.
!pwd

In [ ]:
# 현 폴더의 내용을 보여준다.
!ls

#### 1). 데이터프레임의 내용을 데이터베이스로 옮겨본다: 

#### 데이터프레임을 가져온다:

In [ ]:
os.chdir("../data")
df = pd.read_csv("data_studentlist_en.csv",header="infer")

In [ ]:
nrows = df.shape[0]
my_header = df.columns
df.head(3)

In [ ]:
# 구조를 본다.
df.info()

#### 테이블 내용 가져와 보기:

In [ ]:
# SQLite 데이터 베이스 생성(또는 연결) & Cursor 생성.
conn = sqlite3.connect("student.db")                         # 처음 실행시 신규 생성.
cur = conn.cursor()

In [ ]:
# SQL 테이블 생성.
sql = "CREATE TABLE IF NOT EXISTS studentlist ( name TEXT, gender TEXT, age INT, grade INT, absence TEXT, bloodtype TEXT, height REAL, weight REAL);"
cur.execute(sql);
conn.commit();

In [ ]:
# SQL 테이블 내용 클리어.
sql = "DELETE FROM studentlist;"
cur.execute(sql);
conn.commit();

In [ ]:
# SQL 테이블 내용 확인.
sql = "SELECT * FROM studentlist;"
cur.execute(sql);
res = cur.fetchall();
print(res)

#### 한 행 씩 옮겨 담는다:

In [ ]:
# "studentlist" 테이블에 행 추가.
sql = "INSERT INTO studentlist VALUES ( ?, ?, ?, ?, ?, ?, ?, ? );"      # ?를 placeholder로 사용!
for i in range(nrows):
    x = df.loc[i].values                         # 삽입될 내용.
    x = list(x)
    x[2] = int(x[2])                               # INT64를 INT로 변환.
    x[3] = int(x[3])                               # INT64를 INT로 변환.
    cur.execute(sql,x);                            # 삽입.  
    conn.commit();

In [ ]:
# 테이블 내용 보기.
sql = "SELECT * FROM studentlist"
cur.execute(sql);
res = cur.fetchall()
for x in res:
    print(x)

#### 2). 데이터베이스의 내용을 데이터프레임으로 옮겨본다: 

In [ ]:
# 테이블 내용 가져온다.
sql = "SELECT * FROM studentlist"
cur.execute(sql);
res = cur.fetchall()

In [ ]:
res

In [ ]:
# 새로운 데이터 프레임 생성 & 확인.
df_new = pd.DataFrame(data=res, columns=my_header, index=None)
df_new.head(5)

#### 3). 뒷 정리:

In [ ]:
# 테이블 삭제.
sql = "DROP TABLE studentlist;"
cur.execute(sql);
conn.commit();

In [ ]:
conn.close()